In [1]:
import keras
from keras.models import Sequential
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
import keras.backend as K
K.set_image_data_format('channels_last')

In [4]:
base_model = keras.applications.vgg16.VGG16(weights = 'imagenet',  include_top = False, input_shape=(64,64,3))

In [5]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [9]:
from keras.layers import GlobalAveragePooling2D, Dense, Flatten
from keras.models import Model
from keras.optimizers import Adam

_ = base_model.get_layer("block5_conv3").output
_ = Flatten()(_)
_ = Dense(512, activation='tanh')(_)

# 3 個輸出
predictions = Dense(3, activation='softmax')(_)

# 這是我們的 model
model = Model(inputs=base_model.input, outputs=predictions)

for n in ["block3_conv1","block3_conv2","block3_conv3",\
          "block4_conv1","block4_conv2","block4_conv3",\
          "block5_conv1", "block5_conv2", "block5_conv3"]:
    model.get_layer(n).trainable = True
    
model.compile(optimizer = Adam(lr=0.0001), loss='categorical_crossentropy', metrics=["accuracy"])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [9]:
train_data = np.load('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/ALL_TRAIN_NOSEG_224.npy')
test_data = np.load('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/ALL_TEST_NOSEG_224.npy')
train = pd.read_csv('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/TRAIN.csv')
test = pd.read_csv('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/TEST.csv')
test_id = test.image.values

In [10]:
test_stg2 =\
pd.read_csv('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/TEST_STG2.csv')
test_stg2_data =\
np.load('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/ALL_TEST_STG2_NOSEG_224.npy')
test_stg2_id = test_stg2.image.values

In [12]:
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)

x_train, x_val_train, y_train, y_val_train =\
train_test_split(train_data, train_target, test_size = 0.4, random_state=17)

In [13]:
num_classes = 3
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val_train = keras.utils.to_categorical(y_val_train, num_classes)

In [14]:
x_train = x_train.astype('float32')
x_val_train = x_val_train.astype('float32')

x_train /= 255
x_val_train /= 255

x_test = test_data.astype('float32')
x_test /= 255

In [15]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

In [21]:
# shift = 0.2
datagen = ImageDataGenerator(rotation_range = 180,\
                             horizontal_flip = True,\
                             vertical_flip = True,\
                             width_shift_range = 0.1,\
                             height_shift_range = 0.1,\
                             data_format = "channels_last")
datagen.fit(x_train)

In [22]:
earlystopping = EarlyStopping(monitor='val_loss', patience = 3)

In [23]:
model.fit_generator(datagen.flow(x_train, y_train,\
                                 batch_size = 16, shuffle = True),\
                    epochs = 20,\
                    steps_per_epoch = 500,\
                    verbose = 1,\
                    validation_data = (x_val_train, y_val_train),\
                    callbacks = [earlystopping])

Epoch 1/20
500/500 [==============================] - 184s - loss: 1.0054 - acc: 0.5264 - val_loss: 1.0087 - val_acc: 0.5263

KeyboardInterrupt: 

In [ ]:
pred = model.predict(x_test)